# Importamos Librerias

In [2]:
from _utils import *

In [30]:
# Cargamos los metadatos de CoinGecko
exchanges_metadata = read_json("../../metadata/coingecko/exchanges.json")
categories_metadata = read_json("../../metadata/coingecko/categories.json")

## Obtener la info de las coins de las categorias principales

Obtener las categorias principales (IA, RWA, Memes, Gaming)

In [19]:
len(categories_metadata)

350

In [4]:
# Filtrar las categorías utilizando la expresión regular
regex_categories = create_regex(
    words=["meme", "memes", "game", "gaming", "games", "ai", "real world assets", "rwa", "juego"]
)

# Filtrar las categorías que contienen las palabras clave
categories_filter = list(
    filter(
        lambda categoria: regex_categories.search(categoria["name"].lower()),
        categories_metadata,
    )
)

In [20]:
len(categories_filter)

28

Ahora realizamos las peticiones a la api para obtener la info de las coins de las categorias principales

In [ ]:
coins_info: list[dict] = [] # Lista de diccionarios con la información de las monedas
for category in categories_filter:
    page = 1 # Página de la API
    coin_info:list[dict] = [] # Lista de diccionarios con la información de las monedas
    # Usamos un bucle infinito para obtener todas las monedas de la categoría
    while True:
        try:
            response = fetch_data(api=APIS.COINGECKO,uri=f"coins/markets?vs_currency=usd&category={category["category_id"]}&per_page=250&page={page}",parameters={})
        except Exception as e:
            print(e)
            break
        if not response:
            break
        coin_info.extend(response) # Añadimos la información de las monedas a la lista
        page += 1 # Incrementamos la página
        time.sleep(2) # Esperamos 2 segundos
    print(category)
    category_new = category.copy() # Copiamos la categoría
    category_new["coins"] = coin_info # Añadimos la información de las monedas a la categoría
    coins_info.append(category_new) # Añadimos la información de la categoría a la lista

In [18]:
# Calcular el total de monedas
total_coins = 0
for coin in coins_info:
    total_coins += len(coin["coins"])
total_coins

15662

Estandarizamos los coins

In [22]:
new_coins_info:list[dict] = [] # Lista de diccionarios que contiene la información de las monedas
for category in coins_info:
    for coin in category["coins"]:
        new_coin = coin.copy() # Copiar el diccionario de la moneda
        new_coin["category"] = category["name"] # Agregar el nombre de la categoría
        new_coin["category_id"] = category["category_id"]  # Agregar el id de la categoría
        new_coins_info.append(new_coin) # Agregar el diccionario de la moneda a la lista
coins_info = new_coins_info

In [73]:
len(coins_info)

15662

In [75]:
# Ordenamos la lista de coins en base a market_cap
coins_info = sorted(coins_info, key=lambda x: 0 if x["market_cap"] == None else x["market_cap"], reverse=True)

In [24]:
# Guardamos la información en un archivo JSON
save_json(coins_info, f"../../data/json/coingecko/coins.json")

## Obtener la info de las tickers de los exchanges principales para de ahi poder obtener que monedas se encuentran en estas exchanges

La API de CoinGecko ya nos devuelve de la exchange mas importante a la menos importante

In [31]:
len(exchanges_metadata)

837

In [35]:
# Obtendremos los datos de los 20 principales exchanges
exchanges_centralized_info = []
exchanges_decentralized_info = []

for index, exchange in enumerate(exchanges_metadata):
    # Obtenemos la información del exchange
    exchange_info = fetch_data(api= APIS.COINGECKO, uri=f"exchanges/{exchange["id"]}", parameters={})
    # Agregamos el id del exchange
    exchange_info["id"] = exchange["id"]
    # Verificamos si el exchange es centralizado
    if (exchange_info["centralized"]):
        exchanges_centralized_info.append(exchange_info) # Agregamos la información del exchange a la lista de exchanges centralizados
    # Verificamos si el exchange es descentralizado
    elif (not exchange_info["centralized"] ):
        exchanges_decentralized_info.append(exchange_info)
    time.sleep(2)

El endpoint anterior nos devuelve los tickers limitados a 100, asi ahora llamaremos a otra api para obtener todos los tickers

In [36]:
# Obtner informacion de las transacciones entre monedas que suceden
def get_tickers_by_exchange(exchanges: list[dict]) -> list[dict]:
    exchange_tickers_info: list[dict] = [] # Lista para almacenar la información de los tickers de los exchanges
    for exchange in exchanges:
        page = 1 # variable para la paginacion de la API
        exchange_ticker_info = [] # Lista para almacenar la información de los tickers del exchange
        # Usamos un while para obtener toda la información de los tickers del exchange de cada paginacion 
        while True:
            try:
                response = fetch_data(api= APIS.COINGECKO, uri=f"exchanges/{exchange["id"]}/tickers?page={page}", parameters={})
            except:
                break
            if not response["tickers"]: break # Si no hay tickers en la respuesta, salimos del bucle
            exchange_ticker_info.extend(response["tickers"]) # Agregamos la información de los tickers a la lista
            page += 1 # Incrementamos la variable de paginacion
            time.sleep(2) # Esperamos 2 segundos para hacer la siguiente peticion
        exchange_new = exchange.copy() # Copiamos la información del exchange
        exchange_new["tickers"] = exchange_ticker_info # Agregamos la información de los tickers al exchange
        exchange_tickers_info.append(exchange_new) # Agregamos la información del exchange a la lista de exchanges
    return exchange_tickers_info

In [37]:
# LLamamos a la funcion para obtener la informacion de los tickers de los exchanges
exchanges_centralized_info = get_tickers_by_exchange(exchanges_centralized_info) 
exchanges_decentralized_info = get_tickers_by_exchange(exchanges_decentralized_info) 

In [ ]:
# ordenamos los exchanges en base a su volumen de transacciones
exchanges_centralized_info = sorted(exchanges_centralized_info, key=lambda x: x["trade_volume_24h_btc"], reverse=True)
exchanges_decentralized_info = sorted(exchanges_decentralized_info, key=lambda x: x["trade_volume_24h_btc"], reverse=True)

In [9]:
# Guardamos los exchanges
save_json(exchanges_centralized_info, "../../data/json/coingecko/exchanges_centralized.json")
save_json(exchanges_decentralized_info, "../../data/json/coingecko/exchanges_decentralized.json")

## Obtener la info de las coins en base a los exchanges principales 

In [24]:
len(exchanges_centralized_info), len(exchanges_decentralized_info)

(210, 627)

Creamos una funcion que devuelva ordenada una lista de ids en base a `coins_info`

In [95]:
def get_coins_order_by_marketcap(coins_list: list[str], coins_info: list[dict]):
    coins_order = []  # Lista para almacenar las monedas ordenadas
    for coin in coins_list:
        for coin_info in coins_info:
            if coin_info["id"] == coin:
                coins_order.append(coin_info)  # Agregamos la moneda a la lista
    # Solo devolver las ids
    coins_order = [coin["id"] for coin in coins_order]
    return list(set(coins_order))

Esta funcion generara una lista con la informacion de los ids de las `X` principales exchanges y capado a `X` tokens por cada una

In [104]:
N_EXCHANGES = 50  # Número de exchanges (20 mas importantes)
N_TOKENS = 500  # Número de tokens (500 mas importantes)

def generate_set_ids(exchanges_info):
    coins_id: list[dict] = []
    for exchange in exchanges_info[:N_EXCHANGES]:  # Recorremos las categorias
        coins_list = set()
        for coin in exchange["tickers"]:
            coins_list.add((coin["coin_id"]))
        new_exchange = dict()
        new_exchange["id"] = exchange["id"]
        new_exchange["coins"] = get_coins_order_by_marketcap(
            list(coins_list), coins_info
        )[:N_TOKENS]
        coins_id.append(new_exchange)
    return coins_id

Creamos dos listas para guardar la informacion de las exchanges y las ids que manejan (20 exchanges principales y 500 tokens por cada una)

In [102]:
coins_centralized_id: list[dict] = generate_set_ids(exchanges_centralized_info) # Conjunto para almacenar los id de las monedas
coins_decentralized_id: list[dict] = generate_set_ids(exchanges_decentralized_info) # Conjunto para almacenar los id de las monedas

In [122]:
# Hallar el máximo número de monedas en los exchanges centralizados
max(
    reduce(
        lambda final, exchange: final + [len(exchange["coins"])],
        coins_centralized_id,
        [0],
    )
), max( # Hallar el máximo número de monedas en los exchanges descentralizados
    reduce(
        lambda final, exchange: final + [len(exchange["coins"])],
        coins_decentralized_id,
        [0],
    )
)

(500, 500)

In [123]:
# Cantidad de monedas por tipo exchange
reduce(lambda final, coin: final + len(coin["coins"]), coins_centralized_id, 0), reduce(
    lambda final, coin: final + len(coin["coins"]), coins_decentralized_id, 0
)

(11943, 5537)

Creamos una funcion que valida si el `coin_id` se encuentra dentro de un exchange

In [ ]:
def is_in_set(list_exchange_id: list[dict], coin_id: str) -> list[str]:
    exchanges = []
    for exchange in list_exchange_id:
        if coin_id in exchange["coins"]:
            exchanges.append(exchange["id"])
    return exchanges

Creamos una funcion que añadira el nuevo atributo `exchange_id` y `exchange_centralized` a las coins

In [108]:
def get_coins_by_exchange(
    coins_info: list[dict]
) -> list[dict]:
    new_coins_info = [] # Lista para almacenar la información de los exchanges
    for coin in coins_info:
        new_coin = coin.copy()
        exchanges_c = is_in_set(coins_centralized_id, coin["id"])
        new_coin["exchanges_centralized"] = exchanges_c
        exchanges_d = is_in_set(coins_decentralized_id, coin["id"])
        new_coin["exchanges_decentralized"] = exchanges_d
        if not exchanges_c and not exchanges_d:
            continue
        new_coins_info.append(new_coin)
    return new_coins_info

In [109]:
# Llamamos a la función para obtener la información de las monedas de los exchanges
new_coins_info = get_coins_by_exchange(coins_info)

In [110]:
len(new_coins_info)

7575

In [111]:
# Guardamos la información en un archivo JSON
save_json(new_coins_info, "../../data/json/coingecko/new_coins.json")